## Defining Common Util Functions

In [1]:
import cirq

In [2]:
'''
Takes a qubitString as input and returns a matrix 
that represents the qubit in the standard basis.
Ex: |00> = [1,0,0,0]
'''
def getQubitVector(bitString):
    bitDict = {}
    bitDict['0'] = np.array([1 , 0])
    bitDict['1'] = np.array([0 , 1])
    matrix = [1]
    
    for c in bitString:
        matrix = np.kron(matrix,bitDict[c])
    return matrix

'''
Given a bitstring, return the decimal number 
represented by the bitstring
'''
def getDecimalNo(bitString):
    val = 0
    n = len(bitString)
    for i in range(0,n):
        val = val + (2**i)*(int)(bitString[-i-1])
    return val

'''
Given a value of n, returns all possible bit-strings of size n.
This function will return a list of 2^n bit strings
'''
def getAllPossibleNBitStrings(n):
    if n==1:
        return ['0','1']
    
    children = getAllPossibleNBitStrings(n-1)
    result = []
    for i in children:
        result.append('0'+i)
        result.append('1'+i)
    return result

## Common function object. 
This is the base class and all functions Deutsch-Jozsa (balanced, constant), Bernstein-Vazirani(different a and b values) inhert from this class

In [3]:
import numpy as np

class FunctionObject:
    def __init__(self,fx,n):
        self.__fx = fx
        self.__n = n
        self.__Uf = self.__createUf()
    
    '''
    Apply the function on the given input.
    '''
    def applyFx(self,input):
        return self.__fx(input)
        
    '''
    The N value which signifies the no. of qubits.
    '''
    def getN(self):
        return self.__n
    
    '''
    The Uf matrix which represents the oracle for this function
    '''
    def getUf(self):
        return self.__Uf
    
    '''
    Given the function to execute, and the qubits |x1x2..xn> and the ancilla|b>.
    This function will return a bit vector representing |x1x2..xn> |b+f(x)>
    '''
    def getFunctionResult(self,x,ancilla):
        fx = self.applyFx(x)
        newAncilla = str((((int)(ancilla) + fx)%2))
        result = x+newAncilla
        return getQubitVector(result)
    
    '''
    Using pointer to f(x) and n denoting the no. of qubits,
    it returns an oracle matrix Uf of size 2^(n+1)x2^(n+1) which
    can be used in a Deutsch-Jozsa or Bernstein-Vazirani circuit
    '''
    def __createUf(self):
        Uf = np.zeros((2**(self.__n+1),2**(self.__n+1)))    
    
        #this dictionary represents the correspondence between bit combinations and Uf indices
        indices_dict = {}
        counter = 0

        inputs = getAllPossibleNBitStrings(self.__n+1)

        for i in inputs:
            #input to the function is the first n bits of the elements (bit patterns) from the dictionary
            x = i[0:self.__n]

            #fx represents the output of function f given the input x
            fx = str(self.applyFx(x))

            #b is the last bit of the bit pattern in the dictionary item
            b = i[self.__n]

            #below we have the (f(x) + b) mod 2
            if(b==fx):
                bfx = '0'
            else:
                bfx = '1'
                # print(bfx)

            #the final bit string is the concatenation of the input x and bfx
            result = x + bfx

            #using indices_dict we can now find the index that corresponds to this output
            column = getDecimalNo(result)
            row = getDecimalNo(i)
            #now using the target indiex we can create a bit pattern with all 0s and 1 at the target index position
            Uf[row][column] = 1        
        return Uf
    
    '''
    Given a function and n qubits. This function will verify if the Uf matrix 
    generated for this function matches the expected output. 
    i.e., it checks if Uf|x1x2..xn>|b> = |x1x2..xn> |b+f(x)>
    '''
    def verifyUf(self,debug=False):
        
        UfMatrix = self.getUf()
        if debug:
            print("\nUfMatrix for function \n"+str(UfMatrix)+"\n")

        inputs = getAllPossibleNBitStrings(self.__n+1)
        valid = True

        for i in inputs:
            inputBitVector = getQubitVector(i)

            if debug:
                print("Input BitString {}".format(i))
                print("Input BitVector {}\n".format(inputBitVector))

            functionOutput = self.getFunctionResult(i[0:self.__n],i[self.__n])
            UfOutput = np.matmul(UfMatrix,inputBitVector)

            if debug:
                print("\tUf Output {}".format(UfOutput))
                print("\t F Output {}\n".format(functionOutput))

            if np.array_equal(UfOutput,functionOutput) is not True:
                print("ERROR FOR {}\n".format(i))
                valid=False

        return valid

## Bernstein Vazirani Function Object
A subclass of the function object, this creates a function object for the Bernstein-Vazirani Algorithm.

In [4]:
import random

class BernsteinVaziraniFunction(FunctionObject):

    '''
    Initializes a function object with the passed in a and b values.
    If no a and b values are passed, 
    it will initialize 'a' to a random n-bit string of 0s and 1s 
    and b to 0 or 1.
    '''
    def __init__(self,n,a=None,b=None):
        if a is None:
            self.initRandomAandBValues(n)  
        else:
            self.__a = a
            self.__b = b
        FunctionObject.__init__(self, self.fx, n) 
        
    '''
    Initializes random values for a and b
    '''
    def initRandomAandBValues(self,n):
        self.__b = random.randint(0,1)
        self.__a = ''
        for i in range(0,n):
            self.__a+=str(random.randint(0,1))
        return
        
    '''
    Applies the function f(x) = a*x + b to the inputString
    '''
    def fx(self,inputString):
        dotProduct = 0
        for i in range(0,len(self.__a)):
            dotProduct+= ((int)(inputString[i]))*((int)(self.__a[i]))
        dotProduct = dotProduct % 2
        return (dotProduct + self.__b) % 2
    
    def getA(self):
        return self.__a
    
    def getB(self):
        return self.__b

## Quantum Circuit
The common quantum circuit used for both Deutsch-Jozsa and Berstein-Vazirani Algorithm.

<img src="circuit.png" alt="Circuit" style="width: 300px; float: left"/>

In [ ]:
class Oracle(cirq.Gate):
    def __init__(self, n, UfMatrix, name):
        self.__n=n
        self.__UfMatrix=UfMatrix
        self.__name=name
    def num_qubits(self):
        return self.__n 
    def _unitary_(self):
        #return self.__UfMatrix
        #print("Unitary array", np.squeeze(np.asarray(self.__UfMatrix)))
        return np.squeeze(np.asarray(self.__UfMatrix))
        
    def __str__(self):
        return self.__name

In [ ]:
def runMainCircuit(functionObj,debug=False):
    #create a matrix representing Uf
    UfMatrix = functionObj.getUf()
    
    #for a n bit function, we need n+1 qubits(one ancilla bit)
    n = functionObj.getN()+1
    
    # creating an instance of Uf    
    c=cirq.Circuit()
    
    
    qubits = cirq.LineQubit.range(n)
    # setting last qubit to 1
    c.append(cirq.X(qubits[n-1]))
    
    # adding Hadamard gates to all qubits
    for i in range(0,n):
        c.append([cirq.H(qubits[i])])
    
    
    # creating Uf gate
    uf_bv= Oracle(n, UfMatrix, "UF_BV")
    
    # adding Uf gate
    c.append(uf_bv(*qubits))
    
    # helper bit does not require H gate. Result is treated as trash/ garbage    
    for i in range(0,n):
        c.append([cirq.H(qubits[i])])        
    
    # measurements
    for i in range(0,n):
        c.append(cirq.measure(qubits[i])) 
       
    if debug:
        print(c)
    
    simulator = cirq.Simulator()
    
    result = simulator.run(c, repetitions=1)
    
    if debug:
        print(result)
    
    return result

## Post-Process Quantum Circuit Results

Process results of measurement of the above defined quantum circuit to gain information

* a and b values in Bernstein-Vazirani

In [ ]:
import time

    
'''
Given a function f(x)=a.(x)+b and the number of qubits,
this function uses the quantum circuit to identify the values of a and b
'''
def constructBVFunction(functionObj):
    b = functionObj.applyFx("0"*functionObj.getN())
    a = ""
    start = time.time()
    results = runMainCircuit(functionObj)
    end = time.time()
    
    if len(results.measurements) == 0:
        return None,None,None
    
    timetaken = end-start
    for i in range(0,functionObj.getN()):
        if results.measurements[str(i)][0][0] == True:
            a+='1'
        else:
            a+='0'
    return a,b,timetaken

## Experiments with Quantum Circuits

In [ ]:
'''
Driver that verifies the validity of the quantum circuits for different F configurations.
Here F is randomly initialized as ax+b where a is a bit-string of size n, randomly initialized
and b is randomly initialized to 0 or 1
'''
class BernsteinVaziraniDriver():
    def __init__(self,n):
        self.functionObj = BernsteinVaziraniFunction(n)
        
    def runAndVerifyQuantumCircuit(self,debug=False):
        print("*"*20)
        if debug:
            print("Verified that the created Uf matrix is valid: {}".format(self.functionObj.verifyUf()))

        a,b,circuitRunTime = constructBVFunction(self.functionObj)
        if a is None:
            return
        
        assert a == self.functionObj.getA()
        assert b == self.functionObj.getB()
        print("N : {} , A: {}, B: {}, Time : {}".format(self.functionObj.getN(), a,b, circuitRunTime))
        print("*"*20)

In [ ]:
BernsteinVaziraniDriver(1).runAndVerifyQuantumCircuit()
BernsteinVaziraniDriver(2).runAndVerifyQuantumCircuit()
BernsteinVaziraniDriver(3).runAndVerifyQuantumCircuit()
BernsteinVaziraniDriver(4).runAndVerifyQuantumCircuit()
BernsteinVaziraniDriver(5).runAndVerifyQuantumCircuit()
BernsteinVaziraniDriver(6).runAndVerifyQuantumCircuit()
BernsteinVaziraniDriver(7).runAndVerifyQuantumCircuit()
BernsteinVaziraniDriver(8).runAndVerifyQuantumCircuit()
BernsteinVaziraniDriver(9).runAndVerifyQuantumCircuit()
BernsteinVaziraniDriver(10).runAndVerifyQuantumCircuit()
BernsteinVaziraniDriver(11).runAndVerifyQuantumCircuit()
BernsteinVaziraniDriver(12).runAndVerifyQuantumCircuit()
BernsteinVaziraniDriver(13).runAndVerifyQuantumCircuit()
BernsteinVaziraniDriver(14).runAndVerifyQuantumCircuit()

********************
N : 1 , A: 1, B: 0, Time : 0.006150960922241211
********************
********************
N : 2 , A: 10, B: 1, Time : 0.001856088638305664
********************
********************
N : 3 , A: 101, B: 0, Time : 0.0031642913818359375
********************
********************
N : 4 , A: 0011, B: 1, Time : 0.004128932952880859
********************
********************
N : 5 , A: 01110, B: 0, Time : 0.00420832633972168
********************
********************
N : 6 , A: 000111, B: 1, Time : 0.006457090377807617
********************
********************
N : 7 , A: 1001001, B: 0, Time : 0.01150202751159668
********************
********************
N : 8 , A: 01101110, B: 1, Time : 0.01271200180053711
********************
********************
N : 9 , A: 111001001, B: 0, Time : 0.02702784538269043
********************
********************
N : 10 , A: 0001010111, B: 1, Time : 0.07390999794006348
********************
********************
N : 11 , A: 11110000010, B: 1, Time :

In [ ]:
BernsteinVaziraniDriver(1).runAndVerifyQuantumCircuit()
BernsteinVaziraniDriver(2).runAndVerifyQuantumCircuit()
BernsteinVaziraniDriver(3).runAndVerifyQuantumCircuit()
BernsteinVaziraniDriver(4).runAndVerifyQuantumCircuit()
BernsteinVaziraniDriver(5).runAndVerifyQuantumCircuit()
BernsteinVaziraniDriver(6).runAndVerifyQuantumCircuit()
BernsteinVaziraniDriver(7).runAndVerifyQuantumCircuit()
BernsteinVaziraniDriver(8).runAndVerifyQuantumCircuit()
BernsteinVaziraniDriver(9).runAndVerifyQuantumCircuit()
BernsteinVaziraniDriver(10).runAndVerifyQuantumCircuit()
BernsteinVaziraniDriver(11).runAndVerifyQuantumCircuit()
BernsteinVaziraniDriver(12).runAndVerifyQuantumCircuit()
BernsteinVaziraniDriver(13).runAndVerifyQuantumCircuit()
BernsteinVaziraniDriver(14).runAndVerifyQuantumCircuit()

********************
N : 1 , A: 1, B: 0, Time : 0.005873918533325195
********************
********************
N : 2 , A: 00, B: 1, Time : 0.005497932434082031
********************
********************
N : 3 , A: 011, B: 1, Time : 0.004821062088012695
********************
********************
N : 4 , A: 1010, B: 0, Time : 0.0051348209381103516
********************
********************
N : 5 , A: 11100, B: 1, Time : 0.01431894302368164
********************
********************
N : 6 , A: 000110, B: 0, Time : 0.008998870849609375
********************
********************
N : 7 , A: 1100001, B: 1, Time : 0.012119054794311523
********************
********************
N : 8 , A: 10111011, B: 0, Time : 0.028727054595947266
********************
********************
N : 9 , A: 000001011, B: 0, Time : 0.12543487548828125
********************
********************
N : 10 , A: 1101010101, B: 0, Time : 0.06005406379699707
********************
********************
N : 11 , A: 00101000100, B: 0, Time

In [ ]:
BernsteinVaziraniDriver(1).runAndVerifyQuantumCircuit()
BernsteinVaziraniDriver(2).runAndVerifyQuantumCircuit()
BernsteinVaziraniDriver(3).runAndVerifyQuantumCircuit()
BernsteinVaziraniDriver(4).runAndVerifyQuantumCircuit()
BernsteinVaziraniDriver(5).runAndVerifyQuantumCircuit()
BernsteinVaziraniDriver(6).runAndVerifyQuantumCircuit()
BernsteinVaziraniDriver(7).runAndVerifyQuantumCircuit()
BernsteinVaziraniDriver(8).runAndVerifyQuantumCircuit()
BernsteinVaziraniDriver(9).runAndVerifyQuantumCircuit()
BernsteinVaziraniDriver(10).runAndVerifyQuantumCircuit()
BernsteinVaziraniDriver(11).runAndVerifyQuantumCircuit()
BernsteinVaziraniDriver(12).runAndVerifyQuantumCircuit()
BernsteinVaziraniDriver(13).runAndVerifyQuantumCircuit()
BernsteinVaziraniDriver(14).runAndVerifyQuantumCircuit()

********************
N : 1 , A: 1, B: 0, Time : 0.00666499137878418
********************
********************
N : 2 , A: 11, B: 0, Time : 0.0032529830932617188
********************
********************
N : 3 , A: 110, B: 1, Time : 0.004444122314453125
********************
********************
N : 4 , A: 0010, B: 1, Time : 0.0061681270599365234
********************
********************
N : 5 , A: 11011, B: 1, Time : 0.00694727897644043
********************
********************
N : 6 , A: 001111, B: 1, Time : 0.010620832443237305
********************
********************
N : 7 , A: 0101101, B: 1, Time : 0.016762971878051758
********************
********************
N : 8 , A: 00100110, B: 0, Time : 0.013478994369506836
********************
********************
N : 9 , A: 110110000, B: 0, Time : 0.026797056198120117
********************
********************
N : 10 , A: 0000000010, B: 1, Time : 0.07835125923156738
********************
********************
N : 11 , A: 11101101101, B: 0, Tim

In [ ]:
BernsteinVaziraniDriver(1).runAndVerifyQuantumCircuit()
BernsteinVaziraniDriver(2).runAndVerifyQuantumCircuit()
BernsteinVaziraniDriver(3).runAndVerifyQuantumCircuit()
BernsteinVaziraniDriver(4).runAndVerifyQuantumCircuit()
BernsteinVaziraniDriver(5).runAndVerifyQuantumCircuit()
BernsteinVaziraniDriver(6).runAndVerifyQuantumCircuit()
BernsteinVaziraniDriver(7).runAndVerifyQuantumCircuit()
BernsteinVaziraniDriver(8).runAndVerifyQuantumCircuit()
BernsteinVaziraniDriver(9).runAndVerifyQuantumCircuit()
BernsteinVaziraniDriver(10).runAndVerifyQuantumCircuit()
BernsteinVaziraniDriver(11).runAndVerifyQuantumCircuit()
BernsteinVaziraniDriver(12).runAndVerifyQuantumCircuit()
BernsteinVaziraniDriver(13).runAndVerifyQuantumCircuit()
BernsteinVaziraniDriver(14).runAndVerifyQuantumCircuit()

********************
N : 1 , A: 0, B: 0, Time : 0.005599021911621094
********************
********************
N : 2 , A: 00, B: 0, Time : 0.002863168716430664
********************
********************
N : 3 , A: 101, B: 0, Time : 0.004105091094970703
********************
********************
N : 4 , A: 0100, B: 0, Time : 0.005558967590332031
********************
********************
N : 5 , A: 10000, B: 0, Time : 0.005550861358642578
********************
********************
N : 6 , A: 010010, B: 0, Time : 0.007171154022216797
********************
********************
N : 7 , A: 1000011, B: 1, Time : 0.012198686599731445
********************
********************
N : 8 , A: 00000111, B: 0, Time : 0.01493382453918457
********************
********************
N : 9 , A: 010110011, B: 1, Time : 0.031790971755981445
********************
********************
N : 10 , A: 0111111110, B: 0, Time : 0.06923389434814453
********************
********************
N : 11 , A: 10110001110, B: 0, Time

In [ ]:
BernsteinVaziraniDriver(1).runAndVerifyQuantumCircuit()
BernsteinVaziraniDriver(2).runAndVerifyQuantumCircuit()
BernsteinVaziraniDriver(3).runAndVerifyQuantumCircuit()
BernsteinVaziraniDriver(4).runAndVerifyQuantumCircuit()
BernsteinVaziraniDriver(5).runAndVerifyQuantumCircuit()
BernsteinVaziraniDriver(6).runAndVerifyQuantumCircuit()
BernsteinVaziraniDriver(7).runAndVerifyQuantumCircuit()
BernsteinVaziraniDriver(8).runAndVerifyQuantumCircuit()
BernsteinVaziraniDriver(9).runAndVerifyQuantumCircuit()
BernsteinVaziraniDriver(10).runAndVerifyQuantumCircuit()
BernsteinVaziraniDriver(11).runAndVerifyQuantumCircuit()
BernsteinVaziraniDriver(12).runAndVerifyQuantumCircuit()
BernsteinVaziraniDriver(13).runAndVerifyQuantumCircuit()
BernsteinVaziraniDriver(14).runAndVerifyQuantumCircuit()

********************
N : 1 , A: 0, B: 0, Time : 0.00682377815246582
********************
********************
N : 2 , A: 01, B: 1, Time : 0.004633903503417969
********************
********************
N : 3 , A: 101, B: 0, Time : 0.0031960010528564453
********************
********************
N : 4 , A: 1001, B: 0, Time : 0.0042781829833984375
********************
********************
N : 5 , A: 11011, B: 1, Time : 0.004960060119628906
********************
********************
N : 6 , A: 001010, B: 0, Time : 0.007283926010131836
********************
********************
N : 7 , A: 1110001, B: 0, Time : 0.007884979248046875
********************
********************
N : 8 , A: 00001111, B: 0, Time : 0.0145111083984375
********************
********************
N : 9 , A: 010000000, B: 1, Time : 0.025935888290405273
********************
********************
N : 10 , A: 1010111111, B: 1, Time : 0.07514595985412598
********************
********************
N : 11 , A: 01011011101, B: 1, Time